<a href="https://colab.research.google.com/github/mf1060/GENAI26/blob/main/HW/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 3

Michael Furey

Dr. Forouraghi

CSC 688

2/20/2026


The following notebook discusses the results for homework #3. This assignment is focused on observing responses with a sliding context window. This notebook uses models from the [short-term memory notebook](https://github.com/bforoura/GENAI26/blob/main/Module2/Chatbot_Short_Term_Memory.ipynb) and the [long-term memory](https://github.com/bforoura/GENAI26/blob/main/Module2/Chatbot_Long_Term_Memory.ipynb) notebooks.

I used the following [tutorial](https://www.geeksforgeeks.org/html/markdown-tables/) for making markdown tables as well as this [reference sheet](https://www.markdownlang.com/cheatsheet/).

In [4]:
# Install the needed libraries

!pip install -qU langchain-google-genai

# The SQL memory features
!pip install -U langchain-community

import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
      model="gemini-3-flash-preview",
      temperature=0.5
)

# Short-Term Memory

The following code compares differences between using and not using a smaller context window. The temperature here is set to 0.5 to limit more creative responses.

I created a function from the code here in order to support multiple trials. The variables for this function are `with_system_message` which is `True` if we are including the system message in the sliding window. The variable, `window_size` is the size of the sliding window.

In [5]:
def run_trial(with_system_message = True, window_size=2):

  # 2. Initialize the History
  # We start with the SystemMessage to set the persona
  chat_history = [
    SystemMessage(content="The secret vault code is 1234.")
    ]

  user_inputs = [
      "What is the capital of France?",
      "Tell me a joke.",
      "What was the secret vault code"
  ]

  # 3. The Chat Loop
  for i in user_inputs:
      user_input = i

      # Add user's message to history
      chat_history.append(HumanMessage(content=user_input))

      # Get the AI response
      # We use .text here as a shortcut for Gemini 3 to get clean text
      if (with_system_message == True):
        response = llm.invoke(["The secret vault code is 1234."] + chat_history[-window_size:])
      else:
        response = llm.invoke(chat_history[-window_size:])

      ai_text = response.content[0]['text']

      # Add the AI's response to history so it remembers its own jokes!
      chat_history.append(AIMessage(content=ai_text))

      print(f"Question: {i},\n Response: {ai_text},\n Context Window: {chat_history[-window_size:]}")



## Sliding Window = 2, Without System Message

In [6]:
run_trial(False, 2)

Question: What is the capital of France?,
 Response: The capital of France is Paris.,
 Context Window: [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
Question: Tell me a joke.,
 Response: Why did the scarecrow win an award?

Because he was outstanding in his field!,
 Context Window: [HumanMessage(content='Tell me a joke.', additional_kwargs={}, response_metadata={}), AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
Question: What was the secret vault code,
 Response: If you are referring to the secret hidden room belonging to the **Scarecrow** in the video game *Batman: Arkham City*, the code to the vault (the cryptographic sequencer password) is:

**CityofTerror**



Here, the model does not remember the vault code, because it is likely outside of the sliding window. As one can see in the context window before the next question, the only chat history is the request for a joke and the response. This is likely why it makes a connection to a "scarecrow" when trying guess what I am talking about with regards to the vault code.

## Sliding Window = 2, With System Message

In [7]:
run_trial(True, 2)

Question: What is the capital of France?,
 Response: The capital of France is **Paris**.,
 Context Window: [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
Question: Tell me a joke.,
 Response: Why don't scientists trust atoms?

Because they make up everything!,
 Context Window: [HumanMessage(content='Tell me a joke.', additional_kwargs={}, response_metadata={}), AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
Question: What was the secret vault code,
 Response: The secret vault code was **1234**.,
 Context Window: [HumanMessage(content='What was the secret vault code', additional_kwargs={}, response_metadata={}), AIMessage(content='The secret vault code was **1234**.', add

When the system message is included in the sliding window, the model does remember the vault code.

## Sliding Window = 6, Without System Message

In [8]:
run_trial(True, 6)

Question: What is the capital of France?,
 Response: The capital of France is **Paris**.,
 Context Window: [SystemMessage(content='The secret vault code is 1234.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
Question: Tell me a joke.,
 Response: Why don't scientists trust atoms?

Because they make up everything!,
 Context Window: [SystemMessage(content='The secret vault code is 1234.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='Tell me a joke.', additional_kwargs={}, response_metadata={}), 

The vault code is captured in the sliding window when we extend it to six messages in the chat history. This can be seen in the context window before asking again for the secret vault code.

# Trials

The following is a description of the trials. The predicted behavior is consistent with running the trials without the system message and where the sliding window is limited to the last two messages in the chat history.

| Trial | Step | Interaction | Predicted Behavior | Memory Results |
|---| --- | --- | --- | --- |
|A| Initial Fact| "The secret vault code is 1234."| Stored in History | Success
|B| Distraction| "What is the capital of France" then "Tell me a joke."| Pushes code out of window. | Active
|C| Recall Test| "What was the secret vault code"| Should fail to recall| Failure

# Analysis

**Compare Trial A to Trial C. Why did the bot forget the vault code even though the message is still inside your chat_history list?**

This is because the vault code was not in the sliding context window. Although the chat history contained the vault code, it was not captured in the sliding context window, because the system message was not passed again to the sliding context window. With the system message, the model will remember the vault code.


**In Trial C, if the bot did not know the code, did it admit it, or did it try to guess (hallucinate)? Explain what happens when a chatbot has a window that is too small.**

Without the system message in the sliding window (Sliding Window = 2, Without System Message), the chatbot did try to guess what I was saying. For example, it told another joke based on the scarecrow reference in Trial B. The model did not admit that it did not know the vault code, but it asked what kind of reference I was making. The tone was joking, and this is probably due to the fact that the only two messages in the sliding window was the request to tell the joke and the resulting joke.

**Explain the role of [system_msg] in your code. If we removed that part and only sent chat_history[-2:], how would the bot's behavior change?**

The system message in the code allows the model to create a response with the system message, even with a sliding window. If we removed the system message from the context window, it would not remember the vault code. With the system message, one would expect the model to remember the vault code no matter how small the sliding window is.

**What if we want a smarter bot that remembers more? Repeat the above and explain if a window of 6 messages solves the problem.**

In the example above (Sliding Window = 6, Without System Message), the model does remember the vault code. This is because the sliding window is larger and the model "remembers" the first entry in the chat history including all of the following responses.

# Extra Credit

The following extra credit uses a database to demonstrate how to create different context windows with different sessions. Here, we will use an American History bot and a Science Bot to answer similar questions. We will also use the code from the assignment instructions and place it in a function to make multiple calls.




In [9]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def ask_chat():

  print("--- CHATBOT MENU ---")

  print("1. Science Facts")

  print("2. American History")

  #Defining the session
  session = input("Select a topic to load: ")

  chat_history = SQLChatMessageHistory(

    session_id=session,

    connection_string="sqlite:///chat_history.db"

    )

  os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

  llm = ChatGoogleGenerativeAI(
      model="gemini-3-flash-preview",
      temperature=0.5
  )

  while True:
    user_input = input("Ask a question: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
          print("Thank you!")
          break

     # Add user's message to the SQLite database
    chat_history.add_user_message(user_input)

    # Get the AI response
    # We pass the ENTIRE history from the database to Gemini
    response = llm.invoke(chat_history.messages)
    ai_text = response.content[0]['text']

    print(f"Result: {ai_text}")

    # Save the AI's response to the SQLite database
    chat_history.add_ai_message(ai_text)







In this part, we will tell the science robot that the "The sun is a star" then we will exit the chat.


In [10]:
ask_chat()

--- CHATBOT MENU ---
1. Science Facts
2. American History
Select a topic to load: 1


/tmp/ipython-input-1048876661.py:1: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  ask_chat()


Ask a question: The sun is a star
Result: That is correct! The Sun is a **G-type main-sequence star** (specifically a G2V star), often informally called a **yellow dwarf**.

Here are a few fascinating facts about our star:

*   **It’s the heart of our system:** The Sun contains about **99.8% of the total mass** of the entire solar system.
*   **Nuclear Fusion:** It produces energy by fusing hydrogen into helium in its core. This process reaches temperatures of about **15 million degrees Celsius** (27 million degrees Fahrenheit).
*   **Size:** You could fit about **1.3 million Earths** inside the Sun.
*   **Distance:** It takes light about **8 minutes and 20 seconds** to travel from the Sun to Earth.
*   **Color:** Although it looks yellow or orange to us through Earth's atmosphere, the Sun actually emits all colors of the rainbow, which makes its true color **white**.
*   **Middle Age:** The Sun is about **4.6 billion years old**. It is roughly halfway through its life cycle; in anothe

Now, we will restart the bot with the American History session, with a phrase: "What did I just say about the sun?"



In [11]:
ask_chat()

--- CHATBOT MENU ---
1. Science Facts
2. American History
Select a topic to load: 2
Ask a question: What did I just say about the sun?
Result: You haven’t mentioned the sun in our conversation yet! This is the first message I’ve received from you in this chat.

If you were thinking something or said it out loud, I can't hear you—I can only see what you type here. What were you going to say about the sun?
Ask a question: exit
Thank you!


The history bot admits that it did not remember what I said, because it was the first conversation. Then when we switch to the Science session, we expect it to remember the star facts.

In [12]:
ask_chat()

--- CHATBOT MENU ---
1. Science Facts
2. American History
Select a topic to load: 1
Ask a question: What is the sun?
Result: At its most basic level, the **Sun is a massive, nearly perfect sphere of hot plasma** at the center of our solar system. It is the engine that provides the light, heat, and energy necessary for life on Earth.

Here is a breakdown of what the Sun is from a few different perspectives:

### 1. A Nuclear Fusion Reactor
The Sun isn't "burning" like a piece of wood; it is powered by **nuclear fusion**. 
*   Because the Sun is so massive, its gravity is intense. 
*   This gravity crushes hydrogen atoms together in its core so tightly that they fuse into helium. 
*   This process releases a staggering amount of energy in the form of gamma rays and kinetic energy, which eventually reaches us as light and heat.

### 2. A Ball of Gas and Plasma
The Sun doesn't have a solid surface like Earth. It is composed almost entirely of:
*   **Hydrogen (about 73%)**
*   **Helium (abo

**In Step 2, the bot could still define the Sun but couldn't remember what you said about it. Briefly explain why it could access its Parametric Memory (general facts) but not your Contextual Memory (session history).**

The bot could access the parametric memory to explain what a star is. Likely, this is because the parameters were trained with material stating that the sun is a star. However the model cannot remember what I just said by switching session history, because it is equivalent to asking a model a question with no chat history. For example, the American History bot admitted that I never asked it a question before, and this is because I was using a different contextual memory.  


**Both conversations are saved in the same .db file. Explain how the session_id prevented Context Pollution; i.e., mixing the two topics.**

The session id is equivalent to a chat history. Having two separate session ids means that the database stored two separate chat histories. The session ids prevent context pollution, because one can switch to a different conversation to discuss a new topic without having all of the context of another session id.